### iCluster needs a separate environment due to dependency issues

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import pandas as pd
import sys,os
import random
import copy

import matplotlib.pyplot as plt
import seaborn as sns

from utils.eval import find_best_matches, generate_exprs

from methods import iClusterPlus

from methods.utils import interpret_results, resultsHandler

from pathlib import Path
import multiprocessing as mp


# Settings for simulated data
N_GENES_LIST = ['5', '50', '500']
SCENARIOS = ['A', 'B', 'C']

for METHOD in [iClusterPlus]:
    for SCENARIO in SCENARIOS:
        for N_GENES in N_GENES_LIST:

            #### prep file paths
            # data files
            data_path = f'/local/DESMOND2_data_simulated/simulated/{SCENARIO}'
            FILE = f'{SCENARIO}.n_genes={N_GENES},m=4,std=1,overlap=yes.exprs_z.tsv'
            file_path = os.path.join(data_path, FILE)
            # output folder
            output_path = os.path.join('/home/bba1401/data/unpast_simluated', METHOD.__name__.split('.')[1], SCENARIO, N_GENES)
            # ground truth file
            ground_truth_file = os.path.join(data_path, f'{SCENARIO}.n_genes={N_GENES},m=4,std=1,overlap=yes.biclusters.tsv')

            if SCENARIO == 'A':
                file_path = file_path.replace('overlap=yes', 'overlap=no')
                ground_truth_file= ground_truth_file.replace('overlap=yes', 'overlap=no')
            
            #### prep params
            combinations = METHOD.generate_arg_list(file_path, output_path, ground_truth_file)
            
            #### run algorithm
            with mp.Pool(30) as pool:
                pool.map(METHOD.run_simulated, combinations)



Running /home/bba1401/data/unpast_simluated/iClusterPlus/A/5/random_state=1/n_cluster=9/type=gaussian/n_burnin=200/n_draw=200/lambda_n=null/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1...Running /home/bba1401/data/unpast_simluated/iClusterPlus/A/5/random_state=1/n_cluster=11/type=gaussian/n_burnin=200/n_draw=200/lambda_n=null/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1...Running /home/bba1401/data/unpast_simluated/iClusterPlus/A/5/random_state=1/n_cluster=6/type=gaussian/n_burnin=200/n_draw=200/lambda_n=null/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1...Running /home/bba1401/data/unpast_simluated/iClusterPlus/A/5/random_state=1/n_cluster=5/type=gaussian/n_burnin=200/n_draw=200/lambda_n=null/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1...


Running /home/bba1401/data/unpast_simluated/iClusterPlus/A/5/random_state=1/n_cluster=8/type=gaussian/n_burnin=200/n_draw=200/lambda_n=null/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1...
Running /home/bba1401/data/unpast_simluated/iClust

Loading required package: parallel
Loading required package: parallel
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: parallel
Loading required package: parallel
Loading required package: data.table
Loading required package: parallel
Loading required package: parallel
Loading required package: parallel
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: parallel
Loading required package: parallel
Loading required package: parallel
Loading required package: data.table
Loading required package: parallel
Loading required package: data.table
Loading required package: paral

50  points of lambdas are used to tune parameters.
50  points of lambdas are used to tune parameters.
50  points of lambdas are used to tune parameters.
Begin parallel computation
Begin parallel computation
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin parallel computation
50  points of lambdas are used to tune parameters.
Begin paralle

In [ ]:
for METHOD in [NMF, sparse_PCA]:  # MOFA2
    for SCENARIO in SCENCARIOS:
        for N_GENES in N_GENES_LIST:
            
            combinations = METHOD.generate_arg_list(file_path, output_path, ground_truth_file)
            
            scores = []
            for comb in combinations:
                try:
                    score, runtime = resultsHandler.evaluate_simulated(**comb)
                    scores.append(score)
                except:
                    continue
            max(scores)

In [ ]:
scores = []
for comb in combinations:
    score, runtime = resultsHandler.evaluate_simulated(**comb)
    scores.append(score)
max(scores)